In [2]:
# import libraries
from bs4 import BeautifulSoup
import requests
from urllib.request import (urlopen, urlparse, urlunparse, urlretrieve)
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np 
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
from urllib.request import (Request, urlopen, urlparse, urlunparse, urlretrieve)

print('All libraries loaded ...')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.1.0-py_0 conda-forge
    branca:  0.3.0-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge

branca-0.3.0-p 100% |################################| Time: 0:00:00 323.93 kB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00 373.75 kB/s
altair-2.1.0-p 100% |################################| Time: 0:00:00 556.23 kB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  38.24 MB/s
All libraries loaded ...


In [3]:
json_request = 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/united_states_rankings_2017_limit0_068d27a033ada75528617769a8a4c4c2.json'
response = urlopen(json_request)

# Convert bytes to string type and string type to dict

string = response.read().decode('utf-8')
json_obj = json.loads(string)


In [4]:
univ_df = json_normalize(json_obj['data'])

In [5]:
pd.set_option('max_colwidth', 200)
pd.set_option('max_columns',100)
univ_df.head()

,aliases,apply_link,location,member_level,name,nid,rank,rank_order,record_type,scores_engagement,scores_engagement_rank,scores_environment,scores_environment_rank,scores_outcomes,scores_outcomes_rank,scores_overall,scores_overall_rank,scores_resources,scores_resources_rank,stats_board,stats_fees_oos,stats_salary,subjects_offered,url
0,Stanford University,NaN,California,11,Stanford University,467,1,1,private,17.4,38,7.9,66,39.5,6,92.1,1,27.3,7,"$13,631","$44,757","$83,400","Business & Management,General Engineering,Education,Chemistry,Law,Accounting & Finance,Biological Sciences,Economics & Econometrics,Civil Engineering,Geology, Environmental, Earth & Marine Science...",/world-university-rankings/stanford-university
1,Harvard University,NaN,Massachusetts,0,Harvard University,466,2,2,private,15.2,647,6.8,182,39.6,3,91.6,2,29.9,1,"$14,669","$43,938","$91,300","Other Health,Archaeology,Computer Science,Art, Performing Arts & Design,Geology, Environmental, Earth & Marine Sciences,Politics & International Studies (incl Development Studies),Psychology,Chemi...",/world-university-rankings/harvard-university
2,Massachusetts Institute of Technology,NaN,Massachusetts,0,Massachusetts Institute of Technology,471,3,3,private,15.7,471,7.1,138,39.3,9,91.4,3,29.3,3,"$13,224","$45,016","$90,400","Chemistry,Archaeology,Other Health,Accounting & Finance,Electrical & Electronic Engineering,Mechanical & Aerospace Engineering,Architecture,Communication & Media Studies,Chemical Engineering,Art, ...",/world-university-rankings/massachusetts-institute-technology
3,University of Pennsylvania,NaN,Pennsylvania,0,University of Pennsylvania,480,4,4,private,17.6,20,6.9,169,39.6,4,91.3,4,27.2,8,"$13,464","$47,668","$78,900","Physics & Astronomy,Politics & International Studies (incl Development Studies),Psychology,Sociology,Geography,Mathematics & Statistics,Chemistry,Veterinary Science,Computer Science,Mechanical & A...",/world-university-rankings/university-pennsylvania
4,Columbia University,NaN,New York,0,Columbia University,476,5,5,private,16.8,145,7.8,69,39.6,1,91.1,5,27.1,10,"$12,432","$51,008","$74,000","Medicine & Dentistry,Other Health,Communication & Media Studies,General Engineering,Sociology,Law,Mathematics & Statistics,Accounting & Finance,Languages, Literature & Linguistics,Physics & Astron...",/world-university-rankings/columbia-university


In [6]:
univ_df.shape

(1061, 24)

In [7]:

university_df = univ_df.drop(['aliases','apply_link','member_level','nid','rank','url','scores_engagement_rank','scores_environment_rank','scores_outcomes_rank','scores_resources_rank', 'scores_overall_rank', 'subjects_offered'], axis=1).copy()
university_df.shape

(1061, 12)

In [8]:
# The code was removed by DSX for sharing.

In [9]:
# Get Lat & Lng for each University

latitudes = []
longitudes = []

for index, univ in university_df.iterrows():
    url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&components=country:US&address={}'.format(API_key,univ['name'])
    response = requests.get(url).json()
    try:
        geographical_data = response['results'][0]['geometry']['location'] # get geographical coordinates
    except:
        print('no response for', index, univ['name'])
    latitudes.append(geographical_data['lat'])
    longitudes.append(geographical_data['lng'])
    print('index:  {},  university: {}, city: {}, lat: {}, Lng: {}'.format(index,univ['name'],univ['location'],geographical_data['lat'],geographical_data['lng']))


index:  0,  university: Stanford University, city: California, lat: 37.4274745, Lng: -122.169719
index:  1,  university: Harvard University, city: Massachusetts, lat: 42.3770029, Lng: -71.11666009999999
index:  2,  university: Massachusetts Institute of Technology, city: Massachusetts, lat: 42.360091, Lng: -71.09416
index:  3,  university: University of Pennsylvania, city: Pennsylvania, lat: 39.9522188, Lng: -75.1932137
index:  4,  university: Columbia University, city: New York, lat: 40.8075355, Lng: -73.9625727
index:  5,  university: Yale University, city: Connecticut, lat: 41.3163244, Lng: -72.92234309999999
index:  6,  university: Duke University, city: North Carolina, lat: 36.0014258, Lng: -78.9382286
index:  7,  university: Cornell University, city: New York, lat: 42.4534492, Lng: -76.4735027
index:  8,  university: Princeton University, city: New Jersey, lat: 40.3439888, Lng: -74.6514481
index:  9,  university: Northwestern University, city: Illinois, lat: 42.0564594, Lng: -87.

In [10]:
# Append lat & lng columns to the dataframe


lat_df = pd.DataFrame(latitudes, columns=['Latitude'])
lng_df = pd.DataFrame(longitudes, columns=['Longitude'])

final_df = pd.concat([university_df,lat_df,lng_df],axis=1)

In [11]:
final_df.head()

,location,name,rank_order,record_type,scores_engagement,scores_environment,scores_outcomes,scores_overall,scores_resources,stats_board,stats_fees_oos,stats_salary,Latitude,Longitude
0,California,Stanford University,1,private,17.4,7.9,39.5,92.1,27.3,"$13,631","$44,757","$83,400",37.427475,-122.169719
1,Massachusetts,Harvard University,2,private,15.2,6.8,39.6,91.6,29.9,"$14,669","$43,938","$91,300",42.377003,-71.116660
2,Massachusetts,Massachusetts Institute of Technology,3,private,15.7,7.1,39.3,91.4,29.3,"$13,224","$45,016","$90,400",42.360091,-71.094160
3,Pennsylvania,University of Pennsylvania,4,private,17.6,6.9,39.6,91.3,27.2,"$13,464","$47,668","$78,900",39.952219,-75.193214
4,New York,Columbia University,5,private,16.8,7.8,39.6,91.1,27.1,"$12,432","$51,008","$74,000",40.807536,-73.962573


In [12]:
# save dataset

final_df.to_csv('university_dataset.csv', index=False)


In [13]:
#final_df = pd.read_csv('university_dataset.csv')

In [16]:
# Select the top 200 universities for analysis
df = final_df[:200].copy()
print(df.shape)
df.head()

(200, 14)


,location,name,rank_order,record_type,scores_engagement,scores_environment,scores_outcomes,scores_overall,scores_resources,stats_board,stats_fees_oos,stats_salary,Latitude,Longitude
0,California,Stanford University,1,private,17.4,7.9,39.5,92.1,27.3,"$13,631","$44,757","$83,400",37.427475,-122.169719
1,Massachusetts,Harvard University,2,private,15.2,6.8,39.6,91.6,29.9,"$14,669","$43,938","$91,300",42.377003,-71.116660
2,Massachusetts,Massachusetts Institute of Technology,3,private,15.7,7.1,39.3,91.4,29.3,"$13,224","$45,016","$90,400",42.360091,-71.094160
3,Pennsylvania,University of Pennsylvania,4,private,17.6,6.9,39.6,91.3,27.2,"$13,464","$47,668","$78,900",39.952219,-75.193214
4,New York,Columbia University,5,private,16.8,7.8,39.6,91.1,27.1,"$12,432","$51,008","$74,000",40.807536,-73.962573


In [18]:
# We select 11 most important venue categories (purely subjective) on which to assess overall University allure

venue_cat = ['Office',
'Shop & Service',
'Food',
'Arts & Entertainment',
'Outdoors & Recreation',
'Library',
'College & University',
'Nightlife Spot',
'Residence',
'Government Building',
'Travel & Transport']

# Corresponding Category ID from Foursquare Docs

venue_catid = ['4bf58dd8d48988d124941735',
'4d4b7105d754a06378d81259',
'4d4b7105d754a06374d81259',
'4d4b7104d754a06370d81259',
'4d4b7105d754a06377d81259',
'4bf58dd8d48988d12f941735',
'4d4b7105d754a06372d81259',
'4d4b7105d754a06376d81259',
'4e67e38e036454776db1fb3a',
'4bf58dd8d48988d126941735',
'4d4b7105d754a06379d81259']

cat_name_df = pd.DataFrame(venue_cat, columns=['cat_name'])
cat_id_df =  pd.DataFrame(venue_catid, columns=['cat_id'])
cat_df = pd.concat([cat_name_df,cat_id_df], axis=1)

cat_df

,cat_name,cat_id
0,Office,4bf58dd8d48988d124941735
1,Shop & Service,4d4b7105d754a06378d81259
2,Food,4d4b7105d754a06374d81259
3,Arts & Entertainment,4d4b7104d754a06370d81259
4,Outdoors & Recreation,4d4b7105d754a06377d81259
5,Library,4bf58dd8d48988d12f941735
6,College & University,4d4b7105d754a06372d81259
7,Nightlife Spot,4d4b7105d754a06376d81259
8,Residence,4e67e38e036454776db1fb3a
9,Government Building,4bf58dd8d48988d126941735


In [19]:
# Append Venue Category Columns to the Dataframe & initialize values with NaN

df = pd.concat([df,pd.DataFrame(columns=venue_cat)])

df.head()

,Arts & Entertainment,College & University,Food,Government Building,Latitude,Library,Longitude,Nightlife Spot,Office,Outdoors & Recreation,Residence,Shop & Service,Travel & Transport,location,name,rank_order,record_type,scores_engagement,scores_environment,scores_outcomes,scores_overall,scores_resources,stats_board,stats_fees_oos,stats_salary
0,NaN,NaN,NaN,NaN,37.427475,NaN,-122.169719,NaN,NaN,NaN,NaN,NaN,NaN,California,Stanford University,1,private,17.4,7.9,39.5,92.1,27.3,"$13,631","$44,757","$83,400"
1,NaN,NaN,NaN,NaN,42.377003,NaN,-71.116660,NaN,NaN,NaN,NaN,NaN,NaN,Massachusetts,Harvard University,2,private,15.2,6.8,39.6,91.6,29.9,"$14,669","$43,938","$91,300"
2,NaN,NaN,NaN,NaN,42.360091,NaN,-71.094160,NaN,NaN,NaN,NaN,NaN,NaN,Massachusetts,Massachusetts Institute of Technology,3,private,15.7,7.1,39.3,91.4,29.3,"$13,224","$45,016","$90,400"
3,NaN,NaN,NaN,NaN,39.952219,NaN,-75.193214,NaN,NaN,NaN,NaN,NaN,NaN,Pennsylvania,University of Pennsylvania,4,private,17.6,6.9,39.6,91.3,27.2,"$13,464","$47,668","$78,900"
4,NaN,NaN,NaN,NaN,40.807536,NaN,-73.962573,NaN,NaN,NaN,NaN,NaN,NaN,New York,Columbia University,5,private,16.8,7.8,39.6,91.1,27.1,"$12,432","$51,008","$74,000"


In [20]:
# Plot the Universities on the Map

address = 'US'

geolocator = Nominatim(user_agent="random_string",timeout=5)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of US are {}, {}.'.format(latitude, longitude))


The geograpical coordinates of US are 39.7837304, -100.4458825.


In [27]:
us_map = folium.Map(location=[latitude, longitude], zoom_start=4)
#us_map

In [28]:
# Show all universities on map
# add markers to map

for lat, lng, univ, city in zip(df['Latitude'], df['Longitude'], df['name'], df['location']):
    label = '{}, {}'.format(univ,city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(us_map)  

us_map

In [29]:
# The code was removed by DSX for sharing.

#### Limiting the Foursquare Venue Search - (radius = 2KM, LIMIT = 30/Category) 

In [30]:
# Create function to loop through University locations and fetch count of venues - upto 30 - for each venue category

def venue_category_count(university, city, latitudes, longitudes, radius):
    # for each University ...
    for univ, city, lat, lng in zip(university, city, latitudes, longitudes):
        print("getting venue counts for: {},{}".format(univ,city))            
        # for each Venue Category
        for index, row in cat_df.iterrows():
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}&intent=browse'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT,
                row['cat_id'])
            #Get the results
            #print("category_id: ",row['cat_id'])
            #print("url: ", url)
            try:
                results = requests.get(url).json()['response']['venues']
                venue_count = len(results)
                print("venue count: ",venue_count)
                df.loc[df['name']==univ,row['cat_name']] = venue_count
            except:
                pass
                results = []
                venue_count = len(results)
                print("venue count - zero response: ",venue_count)
                df.loc[df['name']==univ,row['cat_name']] = venue_count
    


In [31]:
# Populate the venue count for each category in the main dataframe

venue_category_count(df['name'], df['location'], df['Latitude'], df['Longitude'], radius=2000)

getting venue counts for: Stanford University,California
venue count:  29
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  6
venue count:  30
venue count:  7
venue count:  27
venue count:  10
venue count:  16
getting venue counts for: Harvard University,Massachusetts
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  19
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  30
getting venue counts for: Massachusetts Institute of Technology,Massachusetts
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  30
getting venue counts for: University of Pennsylvania,Pennsylvania
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  24
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count

In [33]:
#df.head(10)
df.tail(10)

,Arts & Entertainment,College & University,Food,Government Building,Latitude,Library,Longitude,Nightlife Spot,Office,Outdoors & Recreation,Residence,Shop & Service,Travel & Transport,location,name,rank_order,record_type,scores_engagement,scores_environment,scores_outcomes,scores_overall,scores_resources,stats_board,stats_fees_oos,stats_salary
190,4,30,9,0,35.205180,0,-85.917660,6,4,21,2,10,4,Tennessee,Sewanee-The University of the South,191,private,15.9,2.8,20.9,60.7,21.1,"$10,600","$35,202","$39,400"
191,14,30,12,4,42.716635,0,-73.752336,15,10,30,0,30,12,New York,Siena College,192,private,14.8,2.6,25.5,60.5,17.6,"$13,060","$32,293","$54,600"
192,30,30,23,4,36.151988,1,-95.946000,30,30,30,29,25,30,Oklahoma,University of Tulsa,193,private,16.9,5.4,16.8,60.5,21.4,"$10,476","$34,405","$43,400"
193,30,22,30,11,39.739300,2,-84.179967,12,30,30,30,19,30,Ohio,University of Dayton,194,private,17.1,4.1,24.3,60.4,14.9,"$11,840","$37,230","$51,400"
194,30,30,27,23,46.070580,1,-118.330635,30,30,30,13,30,30,Washington,Whitman College,195,private,16.0,3.2,18.4,60.4,22.8,"$11,228","$44,800","$43,400"
195,25,30,30,13,40.941889,1,-90.371867,30,26,30,11,30,30,Illinois,Knox College,196,private,15.4,5.7,20.8,60.3,18.3,"$8,724","$40,497","$41,600"
196,3,30,30,0,41.921177,0,-71.537877,7,20,30,4,30,6,Rhode Island,Bryant University,197,private,15.8,4.2,25.7,60.2,14.5,"$14,102","$38,574","$63,200"
197,2,0,4,1,41.305824,0,-95.083325,0,1,4,0,8,1,Oregon,Lewis & Clark College,198,private,14.2,5.0,22.1,60.2,18.9,"$10,906","$43,382","$42,200"
198,0,0,5,0,47.751074,0,-120.740139,0,0,9,0,4,2,Pennsylvania,Washington & Jefferson College,199,private,14.9,2.6,23.9,60.2,18.8,"$10,666","$41,282","$48,900"
199,30,30,30,23,38.936881,3,-76.998692,30,30,30,19,30,30,District of Columbia,Catholic University of America,200,private,16.3,4.4,16.5,60.1,22.9,"$14,518","$39,726","$54,200"


In [34]:
# Convert numeric columns (dollars, scores) to Floats

df['stats_board'] = df['stats_board'].replace('[\$,]', '', regex=True).astype(float)
df['stats_fees_oos'] = df['stats_fees_oos'].replace('[\$,]', '', regex=True).astype(float)
df['stats_salary'] = df['stats_salary'].replace('[\$,]', '', regex=True).astype(float)
df[['scores_engagement','scores_environment','scores_outcomes','scores_overall','scores_resources']] = df[['scores_engagement','scores_environment','scores_outcomes','scores_overall','scores_resources']].astype(float)


In [36]:
df.dtypes

Arts & Entertainment       int64
College & University       int64
Food                       int64
Government Building        int64
Latitude                 float64
Library                    int64
Longitude                float64
Nightlife Spot             int64
Office                     int64
Outdoors & Recreation      int64
Residence                  int64
Shop & Service             int64
Travel & Transport         int64
location                  object
name                      object
rank_order                object
record_type               object
scores_engagement        float64
scores_environment       float64
scores_outcomes          float64
scores_overall           float64
scores_resources         float64
stats_board              float64
stats_fees_oos           float64
stats_salary             float64
dtype: object

In [38]:
# save the final dataframe

df.to_csv('universities_final_df.csv', index=False)

#### Create a new DataFrame with one hot encoded variables for Private, Public Universities

In [39]:

df_onehot = pd.get_dummies(df['record_type'])
df_onehot[venue_cat] = df[venue_cat]
df_onehot[['scores_engagement','scores_environment','scores_outcomes','scores_resources']] = df[['scores_engagement','scores_environment','scores_outcomes','scores_resources']]
df_onehot[['stats_board','stats_fees_oos','stats_salary']] = df[['stats_board','stats_fees_oos','stats_salary']]
df_onehot[['name']] = df[['name']]


In [40]:
df_onehot

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,name
0,1,0,29,30,30,30,30,6,30,7,27,10,16,17.4,7.9,39.5,27.3,13631.0,44757.0,83400.0,Stanford University
1,1,0,30,30,30,30,30,19,30,30,30,30,30,15.2,6.8,39.6,29.9,14669.0,43938.0,91300.0,Harvard University
2,1,0,30,30,30,30,30,30,30,30,30,30,30,15.7,7.1,39.3,29.3,13224.0,45016.0,90400.0,Massachusetts Institute of Technology
3,1,0,30,30,30,30,30,24,30,30,30,30,30,17.6,6.9,39.6,27.2,13464.0,47668.0,78900.0,University of Pennsylvania
4,1,0,24,30,30,30,30,23,30,30,30,13,30,16.8,7.8,39.6,27.1,12432.0,51008.0,74000.0,Columbia University
5,1,0,30,30,30,20,21,16,30,30,30,30,30,16.8,7.3,39.5,26.8,14000.0,45800.0,70100.0,Yale University
6,1,0,30,30,30,30,19,1,30,30,30,9,30,17.2,6.7,39.6,26.8,13290.0,47243.0,76700.0,Duke University
7,1,0,30,30,30,30,26,4,30,30,30,10,30,17.3,6.5,39.1,27.1,13680.0,47286.0,71500.0,Cornell University
8,1,0,30,30,30,30,25,4,30,30,30,13,30,15.2,6.7,39.6,28.3,13620.0,41820.0,76500.0,Princeton University
9,1,0,30,30,30,30,30,2,30,30,30,24,17,17.1,6.3,38.9,27.3,14389.0,47251.0,64200.0,Northwestern University


#### Scale each column between 0 and 1 using MinMaxScaler

In [41]:
from sklearn import preprocessing

In [42]:
scaler = preprocessing.MinMaxScaler()

In [43]:
# ensure only numeric columns remain in the dataset

columns = df_onehot.columns.tolist()
columns.pop(columns.index('name'))
columns

['private',
 'public',
 'Office',
 'Shop & Service',
 'Food',
 'Arts & Entertainment',
 'Outdoors & Recreation',
 'Library',
 'College & University',
 'Nightlife Spot',
 'Residence',
 'Government Building',
 'Travel & Transport',
 'scores_engagement',
 'scores_environment',
 'scores_outcomes',
 'scores_resources',
 'stats_board',
 'stats_fees_oos',
 'stats_salary']

In [46]:
# Check for NaNs

df_onehot.isnull().values.any()

True

In [49]:
# Check for rows with NaN 

df_onehot[pd.isnull(df_onehot).any(axis=1)]

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,name
132,1,0,12,8,19,15,14,4,30,5,1,2,21,14.3,7.4,23.9,19.0,NaN,37830.0,53900.0,Art Center College of Design


In [57]:
# fill NaN with Mean
pd.set_option('max_rows', 200)

df_onehot['stats_board'] = df_onehot['stats_board'].fillna(df_onehot['stats_board'].mean())

In [58]:
# Check for NaN again

df_onehot[pd.isnull(df_onehot).any(axis=1)]

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,name


In [59]:
# Apply scaler created above

df_onehot[columns] = scaler.fit_transform(df_onehot[columns])
df_onehot.head(10)


,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,name
0,1.0,0.0,0.966667,1.0,1.0,1.000000,1.000000,0.200000,1.0,0.233333,0.9,0.333333,0.517241,0.877551,0.851351,0.995671,0.850575,0.666631,0.864132,0.857914,Stanford University
1,1.0,0.0,1.000000,1.0,1.0,1.000000,1.000000,0.633333,1.0,1.000000,1.0,1.000000,1.000000,0.428571,0.702703,1.000000,1.000000,0.776449,0.846331,1.000000,Harvard University
2,1.0,0.0,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.0,1.000000,1.0,1.000000,1.000000,0.530612,0.743243,0.987013,0.965517,0.623572,0.869762,0.983813,Massachusetts Institute of Technology
3,1.0,0.0,1.000000,1.0,1.0,1.000000,1.000000,0.800000,1.0,1.000000,1.0,1.000000,1.000000,0.918367,0.716216,1.000000,0.844828,0.648963,0.927404,0.776978,University of Pennsylvania
4,1.0,0.0,0.800000,1.0,1.0,1.000000,1.000000,0.766667,1.0,1.000000,1.0,0.433333,1.000000,0.755102,0.837838,1.000000,0.839080,0.539780,1.000000,0.688849,Columbia University
5,1.0,0.0,1.000000,1.0,1.0,0.666667,0.653846,0.533333,1.0,1.000000,1.0,1.000000,1.000000,0.755102,0.770270,0.995671,0.821839,0.705671,0.886802,0.618705,Yale University
6,1.0,0.0,1.000000,1.0,1.0,1.000000,0.576923,0.033333,1.0,1.000000,1.0,0.300000,1.000000,0.836735,0.689189,1.000000,0.821839,0.630554,0.918166,0.737410,Duke University
7,1.0,0.0,1.000000,1.0,1.0,1.000000,0.846154,0.133333,1.0,1.000000,1.0,0.333333,1.000000,0.857143,0.662162,0.978355,0.839080,0.671815,0.919101,0.643885,Cornell University
8,1.0,0.0,1.000000,1.0,1.0,1.000000,0.807692,0.133333,1.0,1.000000,1.0,0.433333,1.000000,0.428571,0.689189,1.000000,0.908046,0.665468,0.800296,0.733813,Princeton University
9,1.0,0.0,1.000000,1.0,1.0,1.000000,1.000000,0.066667,1.0,1.000000,1.0,0.800000,0.551724,0.816327,0.635135,0.969697,0.850575,0.746826,0.918340,0.512590,Northwestern University


### Do KMeans cluster analysis on the processed dataset

#### Expecting Clusters to look something like this:
#####  {High Ranked, Urban, Expensive, High Opportunities}
#####  {High Ranked, Semi-Urban, Less Expensive, Mid Opportunities}
##### {Mid Ranked, Urban, Expensive, High Opportunities}
##### {Mid Ranked, Semi-Urban, Less Expensive, Mid Opportunities}
##### {Mid Ranked, Sparse Pop., Moderately Cheap, Mid/Low Opportunities}
##### {Low Ranked, Urban, Expensive, Mid-to-High Opportunities}
##### {Low Ranked, Urban, Less Expensive, Low Opportunities}
##### {Low Ranked, Urban, Moderately Cheap, Low-to-Mid Opportunities}
##### {Low Ranked, Semi-urban, Cheap, Low Opportunities}

In [60]:
# set number of clusters
kclusters = 9

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_onehot[columns])

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 2, 1,
       0, 1, 1, 1, 2, 0, 2, 1, 4, 1, 0, 2, 4, 1, 1, 4, 0, 2, 0, 2, 4, 2, 4,
       8, 2, 4, 2, 0, 2, 8, 2, 4, 4, 2, 0, 2, 8, 4, 5, 2, 7, 0, 6, 0, 4, 2,
       8, 8, 8, 4, 3, 2, 8, 8, 4, 8, 6, 0, 4, 5, 4, 4, 8, 0, 4, 2, 0, 4, 8,
       8, 4, 7, 2, 7, 0, 2, 0, 0, 4, 8, 8, 4, 4, 4, 2, 4, 4, 8, 4, 8, 2, 8,
       2, 3, 2, 8, 6, 6, 8, 4, 2, 4, 3, 2, 8, 8, 8, 4, 2, 7, 2, 4, 8, 4, 6,
       8, 3, 2, 2, 4, 3, 2, 8, 2, 2, 3, 3, 8, 4, 3, 8, 3, 8, 4, 0, 8, 4, 2,
       4, 4, 2, 3, 2, 4, 4, 4, 6, 8, 2, 8, 2, 2, 8, 6, 4, 8, 6, 6, 6, 3, 2,
       8, 3, 8, 8, 8, 2, 7, 6, 4, 4, 8, 4, 6, 7, 7, 8], dtype=int32)

In [61]:
# Add cluster label to the dataset

df_onehot['Cluster'] = kmeans.labels_

In [62]:
# Inverse the Min Max Scaling

df_onehot[columns] = scaler.inverse_transform(df_onehot[columns])

In [63]:
# Add back the rank & location columns to get a full dataframe

df_onehot[['rank_order','location','Latitude','Longitude']] = df[['rank_order','location','Latitude','Longitude']]

In [64]:
df_onehot.head(10)

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,name,Cluster,rank_order,location,Latitude,Longitude
0,1.0,0.0,29.0,30.0,30.0,30.0,30.0,6.0,30.0,7.0,27.0,10.0,16.0,17.4,7.9,39.5,27.3,13631.0,44757.0,83400.0,Stanford University,1,1,California,37.427475,-122.169719
1,1.0,0.0,30.0,30.0,30.0,30.0,30.0,19.0,30.0,30.0,30.0,30.0,30.0,15.2,6.8,39.6,29.9,14669.0,43938.0,91300.0,Harvard University,1,2,Massachusetts,42.377003,-71.116660
2,1.0,0.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,15.7,7.1,39.3,29.3,13224.0,45016.0,90400.0,Massachusetts Institute of Technology,1,3,Massachusetts,42.360091,-71.094160
3,1.0,0.0,30.0,30.0,30.0,30.0,30.0,24.0,30.0,30.0,30.0,30.0,30.0,17.6,6.9,39.6,27.2,13464.0,47668.0,78900.0,University of Pennsylvania,1,4,Pennsylvania,39.952219,-75.193214
4,1.0,0.0,24.0,30.0,30.0,30.0,30.0,23.0,30.0,30.0,30.0,13.0,30.0,16.8,7.8,39.6,27.1,12432.0,51008.0,74000.0,Columbia University,1,5,New York,40.807536,-73.962573
5,1.0,0.0,30.0,30.0,30.0,20.0,21.0,16.0,30.0,30.0,30.0,30.0,30.0,16.8,7.3,39.5,26.8,14000.0,45800.0,70100.0,Yale University,1,6,Connecticut,41.316324,-72.922343
6,1.0,0.0,30.0,30.0,30.0,30.0,19.0,1.0,30.0,30.0,30.0,9.0,30.0,17.2,6.7,39.6,26.8,13290.0,47243.0,76700.0,Duke University,1,7,North Carolina,36.001426,-78.938229
7,1.0,0.0,30.0,30.0,30.0,30.0,26.0,4.0,30.0,30.0,30.0,10.0,30.0,17.3,6.5,39.1,27.1,13680.0,47286.0,71500.0,Cornell University,1,8,New York,42.453449,-76.473503
8,1.0,0.0,30.0,30.0,30.0,30.0,25.0,4.0,30.0,30.0,30.0,13.0,30.0,15.2,6.7,39.6,28.3,13620.0,41820.0,76500.0,Princeton University,1,9,New Jersey,40.343989,-74.651448
9,1.0,0.0,30.0,30.0,30.0,30.0,30.0,2.0,30.0,30.0,30.0,24.0,17.0,17.1,6.3,38.9,27.3,14389.0,47251.0,64200.0,Northwestern University,1,10,Illinois,42.056459,-87.675267


#### Finally, view these clusters on the map

In [65]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=4)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, nam, cluster, rank in zip(df_onehot['Latitude'], df_onehot['Longitude'], df_onehot['name'], df_onehot['Cluster'], df_onehot['rank_order']):
    label = folium.Popup(str(nam) + '_Rank_' + str(rank) + '_Cluster_' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

####  Interpret Clusters; see if there are cheaper, higher ranked universities or lower ranked but cheaper & better located universities.

In [67]:
# Cluster 1

df_onehot.loc[df_onehot['Cluster']==0].describe()

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,Cluster,Latitude,Longitude
count,18.0,18.0,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.0,18.000000,18.000000
mean,1.0,0.0,18.500000,27.000000,25.388889,25.833333,29.944444,2.055556,29.444444,21.666667,9.944444,7.444444,20.222222,16.161111,5.083333,30.038889,23.377778,12292.944444,45728.500000,55711.111111,0.0,40.315674,-80.175104
std,0.0,0.0,8.473141,5.718906,6.563167,4.630462,0.235702,2.412522,1.916560,7.654295,4.595039,3.853527,7.907967,0.746079,1.478572,4.549873,1.897539,1250.490279,3002.686625,10466.298233,0.0,3.535902,7.733887
min,1.0,0.0,5.000000,14.000000,9.000000,17.000000,29.000000,0.000000,22.000000,5.000000,0.000000,1.000000,7.000000,15.000000,2.200000,23.500000,19.300000,9340.000000,37120.000000,39300.000000,0.0,29.717394,-95.401831
25%,1.0,0.0,13.000000,29.250000,20.250000,21.250000,30.000000,1.000000,30.000000,16.000000,6.000000,4.250000,15.250000,15.450000,4.625000,26.200000,22.425000,11310.500000,45437.000000,48075.000000,0.0,39.705554,-84.216850
50%,1.0,0.0,17.000000,30.000000,30.000000,28.000000,30.000000,1.000000,30.000000,23.500000,11.000000,8.000000,21.500000,16.450000,5.150000,28.950000,23.550000,12563.000000,46005.000000,54650.000000,0.0,41.124313,-78.164254
75%,1.0,0.0,26.250000,30.000000,30.000000,30.000000,30.000000,2.000000,30.000000,27.750000,13.000000,10.500000,26.000000,16.700000,6.150000,32.500000,24.225000,13145.250000,48016.000000,61875.000000,0.0,42.608112,-73.738625
max,1.0,0.0,30.000000,30.000000,30.000000,30.000000,30.000000,11.000000,30.000000,30.000000,18.000000,13.000000,30.000000,17.700000,7.200000,39.200000,27.400000,13960.000000,48682.000000,75200.000000,0.0,44.461365,-71.305928


In [69]:
df_onehot.loc[df_onehot['Cluster']==0]

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,name,Cluster,rank_order,location,Latitude,Longitude
14,1.0,0.0,21.0,30.0,24.0,28.0,30.0,1.0,28.0,14.0,10.0,4.0,26.0,16.8,6.0,39.2,26.7,13839.0,48108.0,67400.0,Dartmouth College,0,15,New Hampshire,43.704441,-72.288693
20,1.0,0.0,13.0,30.0,30.0,30.0,30.0,11.0,30.0,27.0,16.0,12.0,25.0,16.7,7.0,35.5,27.4,13400.0,40566.0,62200.0,Rice University,0,21,Texas,29.717394,-95.401831
23,1.0,0.0,7.0,30.0,20.0,30.0,30.0,2.0,30.0,20.0,11.0,3.0,15.0,15.7,6.6,37.5,24.6,12760.0,48310.0,56100.0,Williams College,0,24,Massachusetts,42.712958,-73.203640
28,1.0,0.0,30.0,18.0,30.0,17.0,30.0,3.0,30.0,5.0,6.0,9.0,20.0,16.5,7.2,34.5,24.4,13960.0,45078.0,56600.0,Wellesley College,0,29,Massachusetts,42.293573,-71.305928
33,1.0,0.0,21.0,29.0,21.0,22.0,30.0,1.0,30.0,21.0,18.0,6.0,30.0,17.7,6.6,30.9,24.6,13550.0,46060.0,48900.0,Swarthmore College,0,34,Pennsylvania,39.904035,-75.352317
39,1.0,0.0,24.0,30.0,17.0,30.0,30.0,1.0,30.0,30.0,10.0,8.0,26.0,16.7,5.8,32.2,23.5,12366.0,47736.0,47800.0,Carleton College,0,40,Minnesota,44.461365,-93.155826
41,1.0,0.0,15.0,30.0,30.0,20.0,30.0,2.0,30.0,19.0,6.0,7.0,10.0,16.8,4.9,32.6,23.7,13116.0,46044.0,54200.0,Middlebury College,0,42,Vermont,44.008260,-73.177315
50,1.0,0.0,27.0,30.0,18.0,30.0,30.0,2.0,30.0,30.0,13.0,8.0,24.0,16.7,3.8,31.2,23.0,11642.0,48498.0,68900.0,Bucknell University,0,51,Pennsylvania,40.954687,-76.883551
57,1.0,0.0,5.0,30.0,30.0,21.0,30.0,1.0,30.0,26.0,6.0,1.0,7.0,16.0,5.2,28.7,23.6,11970.0,48175.0,60900.0,Colgate University,0,58,New York,42.816615,-75.540176
64,1.0,0.0,30.0,30.0,30.0,28.0,30.0,4.0,30.0,26.0,13.0,2.0,16.0,15.4,4.6,29.2,22.5,12769.0,45377.0,55100.0,Davidson College,0,65,North Carolina,35.500802,-80.844673


In [70]:
# Cluster 2
df_onehot.loc[df_onehot['Cluster']==1].describe()

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,Cluster,Latitude,Longitude
count,27.0,27.0,27.000000,27.000000,27.0,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.0,27.000000,27.000000
mean,1.0,0.0,29.074074,29.592593,30.0,28.185185,27.592593,9.629630,29.370370,28.148148,28.481481,22.000000,27.148148,16.544444,6.729630,37.681481,26.300000,13656.481481,46347.259259,69381.481481,1.0,39.908247,-82.366765
std,0.0,0.0,2.730538,1.926720,0.0,3.961711,4.040747,7.899115,3.271652,5.141862,3.203008,8.260192,5.164254,0.903554,0.822667,2.502620,1.909088,981.012482,2086.737711,10821.596535,0.0,2.828849,14.654872
min,1.0,0.0,18.000000,20.000000,30.0,16.000000,19.000000,1.000000,13.000000,7.000000,20.000000,8.000000,14.000000,14.100000,4.300000,31.700000,21.200000,11994.000000,41820.000000,43500.000000,1.0,33.792519,-122.169719
25%,1.0,0.0,30.000000,30.000000,30.0,30.000000,25.500000,4.000000,30.000000,30.000000,30.000000,14.000000,27.000000,16.050000,6.350000,37.350000,25.250000,13071.000000,44882.500000,61650.000000,1.0,38.778199,-86.518997
50%,1.0,0.0,30.000000,30.000000,30.0,30.000000,30.000000,7.000000,30.000000,30.000000,30.000000,23.000000,30.000000,16.800000,6.800000,38.900000,26.600000,13620.000000,46664.000000,67800.000000,1.0,40.807536,-76.620518
75%,1.0,0.0,30.000000,30.000000,30.0,30.000000,30.000000,14.000000,30.000000,30.000000,30.000000,30.000000,30.000000,17.200000,7.200000,39.450000,27.150000,14230.500000,47551.000000,76200.000000,1.0,42.187278,-72.779789
max,1.0,0.0,30.000000,30.000000,30.0,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,17.600000,7.900000,39.600000,29.900000,16782.000000,51008.000000,91300.000000,1.0,42.453449,-71.094160


In [71]:
df_onehot.loc[df_onehot['Cluster']==1]

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,name,Cluster,rank_order,location,Latitude,Longitude
0,1.0,0.0,29.0,30.0,30.0,30.0,30.0,6.0,30.0,7.0,27.0,10.0,16.0,17.4,7.9,39.5,27.3,13631.0,44757.0,83400.0,Stanford University,1,1,California,37.427475,-122.169719
1,1.0,0.0,30.0,30.0,30.0,30.0,30.0,19.0,30.0,30.0,30.0,30.0,30.0,15.2,6.8,39.6,29.9,14669.0,43938.0,91300.0,Harvard University,1,2,Massachusetts,42.377003,-71.116660
2,1.0,0.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,15.7,7.1,39.3,29.3,13224.0,45016.0,90400.0,Massachusetts Institute of Technology,1,3,Massachusetts,42.360091,-71.094160
3,1.0,0.0,30.0,30.0,30.0,30.0,30.0,24.0,30.0,30.0,30.0,30.0,30.0,17.6,6.9,39.6,27.2,13464.0,47668.0,78900.0,University of Pennsylvania,1,4,Pennsylvania,39.952219,-75.193214
4,1.0,0.0,24.0,30.0,30.0,30.0,30.0,23.0,30.0,30.0,30.0,13.0,30.0,16.8,7.8,39.6,27.1,12432.0,51008.0,74000.0,Columbia University,1,5,New York,40.807536,-73.962573
5,1.0,0.0,30.0,30.0,30.0,20.0,21.0,16.0,30.0,30.0,30.0,30.0,30.0,16.8,7.3,39.5,26.8,14000.0,45800.0,70100.0,Yale University,1,6,Connecticut,41.316324,-72.922343
6,1.0,0.0,30.0,30.0,30.0,30.0,19.0,1.0,30.0,30.0,30.0,9.0,30.0,17.2,6.7,39.6,26.8,13290.0,47243.0,76700.0,Duke University,1,7,North Carolina,36.001426,-78.938229
7,1.0,0.0,30.0,30.0,30.0,30.0,26.0,4.0,30.0,30.0,30.0,10.0,30.0,17.3,6.5,39.1,27.1,13680.0,47286.0,71500.0,Cornell University,1,8,New York,42.453449,-76.473503
8,1.0,0.0,30.0,30.0,30.0,30.0,25.0,4.0,30.0,30.0,30.0,13.0,30.0,15.2,6.7,39.6,28.3,13620.0,41820.0,76500.0,Princeton University,1,9,New Jersey,40.343989,-74.651448
9,1.0,0.0,30.0,30.0,30.0,30.0,30.0,2.0,30.0,30.0,30.0,24.0,17.0,17.1,6.3,38.9,27.3,14389.0,47251.0,64200.0,Northwestern University,1,10,Illinois,42.056459,-87.675267


In [72]:
# Cluster 3

df_onehot.loc[df_onehot['Cluster']==2].describe()

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,Cluster,Latitude,Longitude
count,40.0,40.0,40.000000,40.000000,40.00000,40.00000,40.000000,40.000000,40.000000,40.0000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.0,40.000000,40.000000
mean,0.0,1.0,27.225000,28.250000,29.85000,28.57500,27.150000,4.600000,29.750000,28.3000,27.350000,17.500000,27.900000,16.527500,6.170000,29.880000,16.585000,11025.825000,30685.300000,53737.500000,2.0,38.706765,-90.941677
std,0.0,0.0,5.327661,4.137074,0.80224,2.94294,4.865103,3.953577,1.126601,3.2597,5.351971,8.174381,4.650889,0.807334,1.611847,5.042801,2.463742,1729.009556,5064.514325,6848.083674,0.0,4.377518,16.240434
min,0.0,1.0,11.000000,16.000000,25.00000,19.00000,13.000000,0.000000,24.000000,19.0000,10.000000,5.000000,14.000000,14.000000,3.000000,20.800000,12.500000,8270.000000,20810.000000,42700.000000,2.0,29.643632,-122.303520
25%,0.0,1.0,27.000000,30.000000,30.00000,29.75000,24.750000,1.750000,30.000000,28.7500,29.000000,10.000000,29.500000,16.275000,4.725000,26.200000,14.925000,9932.500000,27888.250000,48975.000000,2.0,35.507521,-99.612938
50%,0.0,1.0,30.000000,30.000000,30.00000,30.00000,30.000000,3.000000,30.000000,30.0000,30.000000,16.500000,30.000000,16.750000,5.850000,28.900000,16.500000,10716.500000,30086.000000,53150.000000,2.0,39.879297,-85.454615
75%,0.0,1.0,30.000000,30.000000,30.00000,30.00000,30.000000,7.250000,30.000000,30.0000,30.000000,23.500000,30.000000,17.100000,7.425000,34.150000,18.100000,11883.500000,33534.250000,56925.000000,2.0,41.652686,-78.762251
max,0.0,1.0,30.000000,30.000000,30.00000,30.00000,30.000000,14.000000,30.000000,30.0000,30.000000,30.000000,30.000000,17.600000,9.000000,38.900000,22.200000,15830.000000,43377.000000,75200.000000,2.0,47.655335,-72.253980


In [73]:
df_onehot.loc[df_onehot['Cluster']==2]

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,name,Cluster,rank_order,location,Latitude,Longitude
21,0.0,1.0,20.0,30.0,30.0,27.0,30.0,2.0,30.0,30.0,19.0,25.0,15.0,17.4,6.0,38.9,22.2,10246.0,43377.0,57600.0,University of Michigan,2,22,Michigan,42.278044,-83.738224
27,0.0,1.0,17.0,30.0,30.0,24.0,30.0,4.0,30.0,30.0,30.0,28.0,30.0,16.5,8.9,37.3,20.0,14511.0,35583.0,59400.0,"University of California, Los Angeles",2,28,California,34.068921,-118.445181
29,0.0,1.0,30.0,16.0,30.0,19.0,16.0,3.0,30.0,23.0,30.0,23.0,30.0,16.9,5.3,37.4,21.7,10592.0,33418.0,50700.0,University of North Carolina at Chapel Hill,2,30,North Carolina,35.904912,-79.046913
34,0.0,1.0,30.0,30.0,30.0,30.0,30.0,13.0,30.0,30.0,22.0,30.0,26.0,16.8,8.4,36.3,18.1,15830.0,35850.0,61700.0,"University of California, Berkeley",2,35,California,37.870921,-122.268803
40,0.0,1.0,30.0,30.0,30.0,30.0,30.0,2.0,30.0,30.0,30.0,7.0,30.0,17.2,5.6,34.1,21.3,10030.0,28804.0,53400.0,Purdue University,2,41,Indiana,40.423705,-86.921195
42,0.0,1.0,30.0,30.0,30.0,30.0,20.0,0.0,30.0,30.0,30.0,13.0,30.0,17.1,8.4,33.5,18.3,14316.0,36774.0,56700.0,"University of California, Davis",2,43,California,38.537381,-121.752801
44,0.0,1.0,30.0,30.0,30.0,30.0,26.0,6.0,30.0,30.0,30.0,19.0,30.0,17.1,7.4,37.2,15.4,10848.0,32621.0,56600.0,University of Illinois at Urbana-Champaign,2,45,Illinois,40.109888,-88.228383
47,0.0,1.0,16.0,30.0,30.0,30.0,23.0,3.0,30.0,30.0,26.0,12.0,23.0,15.1,8.6,34.3,17.9,12254.0,36305.0,59300.0,"University of California, San Diego",2,48,California,32.880060,-117.234014
49,0.0,1.0,30.0,30.0,30.0,30.0,23.0,4.0,30.0,19.0,30.0,9.0,30.0,15.5,4.9,36.6,17.8,10052.0,42394.0,59300.0,University of Virginia,2,50,Virginia,38.033553,-78.507977
51,0.0,1.0,30.0,30.0,30.0,30.0,30.0,10.0,30.0,30.0,30.0,30.0,30.0,15.3,7.1,36.4,15.5,10434.0,30698.0,74200.0,Georgia Institute of Technology,2,52,Georgia,33.775618,-84.396285


In [74]:
# Cluster 4

df_onehot.loc[df_onehot['Cluster']==3].describe()

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,Cluster,Latitude,Longitude
count,12.0,12.0,12.000000,12.000000,12.000000,12.000000,12.000000,12.00000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.0,12.000000,12.000000
mean,1.0,0.0,16.583333,26.500000,23.000000,24.083333,29.333333,1.25000,29.166667,25.666667,21.000000,7.166667,18.583333,16.308333,3.250000,24.508333,19.816667,10758.750000,40638.083333,50333.333333,3.0,39.817253,-83.156573
std,0.0,0.0,7.204271,6.626394,6.480741,5.107184,2.309401,1.05529,2.886751,6.485415,6.928203,4.174236,6.126816,0.814034,0.860761,0.857542,1.578741,891.716743,6151.263587,4662.877249,0.0,3.283649,6.448393
min,1.0,0.0,3.000000,12.000000,9.000000,14.000000,22.000000,0.00000,20.000000,7.000000,10.000000,2.000000,8.000000,15.200000,2.100000,22.900000,17.100000,9110.000000,28050.000000,44900.000000,3.0,34.923700,-93.968777
25%,1.0,0.0,11.500000,27.750000,21.250000,21.500000,30.000000,0.75000,30.000000,24.500000,16.750000,5.000000,14.000000,15.550000,2.725000,24.250000,19.100000,10308.500000,39175.500000,46700.000000,3.0,37.207589,-87.125715
50%,1.0,0.0,18.000000,30.000000,23.000000,24.000000,30.000000,1.00000,30.000000,28.000000,20.000000,6.500000,19.000000,16.400000,3.100000,24.600000,19.700000,10795.000000,41636.000000,49750.000000,3.0,40.478908,-82.187777
75%,1.0,0.0,22.250000,30.000000,29.250000,28.500000,30.000000,2.00000,30.000000,30.000000,27.000000,8.000000,23.250000,17.125000,3.650000,25.025000,20.450000,11278.000000,45021.000000,52275.000000,3.0,41.323099,-77.402035
max,1.0,0.0,26.000000,30.000000,30.000000,30.000000,30.000000,3.00000,30.000000,30.000000,30.000000,16.000000,27.000000,17.400000,4.900000,25.800000,23.300000,12126.000000,47908.000000,60300.000000,3.0,44.461663,-75.455859


In [75]:
df_onehot.loc[df_onehot['Cluster']==3]

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,name,Cluster,rank_order,location,Latitude,Longitude
73,1.0,0.0,23.0,30.0,23.0,23.0,30.0,1.0,30.0,28.0,17.0,2.0,14.0,17.1,4.9,25.5,23.3,10790.0,46680.0,60300.0,University of Richmond,3,74,Virginia,37.575868,-77.540974
116,1.0,0.0,26.0,30.0,22.0,23.0,30.0,3.0,30.0,23.0,19.0,16.0,11.0,16.4,4.6,24.6,20.9,10250.0,43350.0,45400.0,The College of Wooster,3,117,Ohio,40.811427,-81.937284
125,1.0,0.0,10.0,30.0,30.0,28.0,30.0,1.0,20.0,27.0,30.0,5.0,22.0,16.4,2.5,25.8,21.0,11204.0,44668.0,47100.0,Furman University,3,126,South Carolina,34.923700,-82.438269
139,1.0,0.0,13.0,30.0,29.0,26.0,30.0,1.0,30.0,25.0,26.0,6.0,17.0,16.5,3.3,24.6,19.6,9110.0,40400.0,49500.0,Gustavus Adolphus College,3,140,Minnesota,44.322162,-93.968777
143,1.0,0.0,23.0,30.0,19.0,30.0,30.0,3.0,30.0,28.0,18.0,6.0,23.0,15.6,3.2,24.4,20.3,9500.0,41700.0,44900.0,St Olaf College,3,144,Minnesota,44.461663,-93.182678
148,1.0,0.0,12.0,30.0,23.0,25.0,30.0,1.0,30.0,30.0,23.0,14.0,24.0,15.4,3.6,24.3,19.8,12126.0,47908.0,48600.0,Hobart William Smith Colleges,3,149,New York,42.858117,-76.985216
149,1.0,0.0,22.0,12.0,9.0,30.0,30.0,0.0,30.0,30.0,16.0,2.0,21.0,17.2,2.1,24.7,19.2,11920.0,35652.0,53700.0,Butler University,3,150,Indiana,39.841124,-86.171191
152,1.0,0.0,22.0,15.0,15.0,22.0,30.0,2.0,30.0,7.0,30.0,8.0,27.0,15.2,2.3,25.0,20.3,10328.0,41572.0,51000.0,Rhodes College,3,153,Tennessee,35.155084,-89.989287
154,1.0,0.0,3.0,30.0,24.0,14.0,22.0,1.0,30.0,22.0,21.0,7.0,8.0,17.3,2.8,23.1,19.5,10910.0,28050.0,56200.0,Ohio Northern University,3,155,Ohio,40.766306,-83.832499
164,1.0,0.0,15.0,30.0,30.0,30.0,30.0,0.0,30.0,30.0,30.0,7.0,16.0,17.4,2.9,22.9,18.8,10667.0,31247.0,45500.0,Elon University,3,165,North Carolina,36.102753,-79.502357


In [76]:
# Cluster 5

df_onehot.loc[df_onehot['Cluster']==4].describe()

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,Cluster,Latitude,Longitude
count,43.0,43.0,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000,43.0,43.000000,43.000000
mean,1.0,0.0,29.325581,28.953488,29.348837,28.302326,26.139535,2.790698,28.255814,27.209302,26.441860,9.767442,28.790698,16.127907,5.148837,25.667442,20.881395,12261.000000,42189.302326,54402.325581,4.0,38.630396,-91.573308
std,0.0,0.0,2.466161,2.627270,2.091749,4.512073,5.621309,2.454909,3.374223,5.501334,6.299291,4.555673,3.454177,0.801286,1.577426,3.887937,2.823815,1738.465018,8023.452702,7753.201746,0.0,4.886466,18.185683
min,1.0,0.0,17.000000,19.000000,21.000000,10.000000,11.000000,0.000000,19.000000,11.000000,11.000000,2.000000,14.000000,14.000000,1.600000,16.800000,13.700000,7330.000000,5000.000000,35700.000000,4.0,25.719168,-122.630372
25%,1.0,0.0,30.000000,30.000000,30.000000,30.000000,23.500000,1.000000,29.000000,28.000000,27.000000,5.000000,30.000000,15.600000,3.950000,23.500000,18.350000,11260.000000,40578.500000,49850.000000,4.0,34.902348,-108.305606
50%,1.0,0.0,30.000000,30.000000,30.000000,30.000000,30.000000,2.000000,30.000000,30.000000,30.000000,10.000000,30.000000,16.200000,5.300000,25.500000,21.400000,12582.000000,44350.000000,53600.000000,4.0,39.836207,-88.859591
75%,1.0,0.0,30.000000,30.000000,30.000000,30.000000,30.000000,4.000000,30.000000,30.000000,30.000000,13.000000,30.000000,16.650000,6.400000,27.550000,23.200000,13412.500000,47264.000000,57950.000000,4.0,41.963019,-76.064922
max,1.0,0.0,30.000000,30.000000,30.000000,30.000000,30.000000,11.000000,30.000000,30.000000,30.000000,19.000000,30.000000,18.000000,7.800000,33.200000,25.000000,14820.000000,49570.000000,68700.000000,4.0,47.261764,-69.963997


In [77]:
df_onehot.loc[df_onehot['Cluster']==4]

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,name,Cluster,rank_order,location,Latitude,Longitude
31,1.0,0.0,30.0,30.0,30.0,30.0,23.0,2.0,30.0,28.0,30.0,9.0,30.0,16.4,7.6,31.4,25.0,14700.0,45832.0,52000.0,Pomona College,4,32,California,34.097743,-117.711803
35,1.0,0.0,30.0,30.0,30.0,30.0,21.0,4.0,30.0,30.0,30.0,8.0,30.0,16.0,5.8,33.2,24.7,14350.0,47214.0,56100.0,Haverford College,4,36,Pennsylvania,40.009348,-75.305656
38,1.0,0.0,30.0,30.0,30.0,30.0,14.0,3.0,30.0,30.0,30.0,4.0,30.0,15.1,7.0,31.8,24.3,14350.0,45540.0,52000.0,Bryn Mawr College,4,39,Pennsylvania,40.028106,-75.315234
43,1.0,0.0,30.0,30.0,30.0,30.0,16.0,0.0,30.0,30.0,30.0,5.0,30.0,17.5,7.8,29.8,22.1,12684.0,44350.0,53400.0,University of Miami,4,44,Florida,25.719168,-80.277125
45,1.0,0.0,30.0,25.0,30.0,30.0,30.0,1.0,30.0,30.0,15.0,12.0,30.0,16.4,5.1,31.3,24.0,12760.0,46808.0,55000.0,Bowdoin College,4,46,Maine,43.907693,-69.963997
48,1.0,0.0,30.0,30.0,30.0,30.0,25.0,2.0,30.0,30.0,30.0,11.0,30.0,16.7,6.6,27.7,24.1,14820.0,47395.0,66400.0,Claremont McKenna College,4,49,California,34.101777,-117.707922
54,1.0,0.0,30.0,30.0,21.0,30.0,30.0,3.0,23.0,30.0,30.0,4.0,30.0,16.2,4.3,30.6,23.0,12638.0,46200.0,60400.0,Wake Forest University,4,55,North Carolina,36.135250,-80.276342
55,1.0,0.0,30.0,30.0,30.0,10.0,27.0,10.0,30.0,19.0,30.0,13.0,30.0,16.9,4.9,27.4,24.8,12556.0,48306.0,52600.0,Tulane University,4,56,Louisiana,29.940348,-90.120728
60,1.0,0.0,30.0,22.0,30.0,30.0,27.0,5.0,28.0,30.0,30.0,18.0,30.0,16.3,5.7,26.8,24.7,13708.0,46960.0,55900.0,University of Rochester,4,61,New York,43.130553,-77.626003
67,1.0,0.0,30.0,30.0,30.0,30.0,30.0,4.0,30.0,30.0,18.0,14.0,30.0,15.8,4.9,33.1,17.6,13186.0,47436.0,66800.0,Boston College,4,68,Massachusetts,42.335549,-71.168495


In [ ]:
# Cl